In [1]:
import pandas as pd
import numpy as np
import os
from sklearn.model_selection import train_test_split
import shutil
from tqdm import tqdm

In [2]:
data_path = '/root/mmaction/data/punch101'
annotation_path = data_path + '/annotations'
video_path = data_path + '/videos'
raw_path = data_path + '/rawframes'
os.makedirs(annotation_path, exist_ok=True)
os.makedirs(video_path, exist_ok=True)
os.makedirs(raw_path, exist_ok=True)

In [3]:
classes = {
    0:'NoPunch',
    1:'ViolentPunchContact',
    2:'ViolentPunchNoContact',
    3:'NonViolentPunchContact',
    4:'NonViolentPunchNoContact'
}

In [4]:
for cls in classes.values():
    os.makedirs(os.path.join(video_path, cls), exist_ok=True)
    os.makedirs(os.path.join(raw_path, cls), exist_ok=True)

In [5]:
labeled_vids = pd.read_csv('/root/bittah-ninja/ground_truth.csv')
labeled_vids.groupby('class').size()

class
-1     144
 0    3783
 1    1049
 2      52
 3     266
 4     147
dtype: int64

In [6]:
new_files = []
for file in labeled_vids.clip_title:
    newfile = ''.join(file.split('.mp4')) + '_scaled.mp4'
    new_files.append(newfile)
labeled_vids['clip_title'] = new_files
labeled_vids.rename(columns={'class':'label'}, inplace=True)
labeled_vids = labeled_vids.loc[labeled_vids.label != -1]
labeled_vids = labeled_vids.loc[labeled_vids.label != 99]
labeled_vids.groupby('label').size()

label
0    3783
1    1049
2      52
3     266
4     147
dtype: int64

In [7]:
train, test, train_labels, test_labels = train_test_split(labeled_vids.clip_title, labeled_vids.label, test_size=0.2)

In [8]:
def createAnnotationsList(filenames, labels, output_name):
    with open(os.path.join(annotation_path, output_name), 'w') as f:
        annotations = []
        for name, cls in zip(filenames, labels):
            if cls == -1:
                continue
            filename = os.path.join(classes[cls], name)
            annotations.append(f'{filename} {cls}')
        f.write('\n'.join(annotations))
    return annotations

In [9]:
train_annotations = createAnnotationsList(train, train_labels, 'train_list.txt')
test_annotations = createAnnotationsList(test, test_labels, 'test_list.txt')

In [10]:
annotations = train_annotations.copy()
annotations.extend(test_annotations)
len(annotations), len(train_annotations), len(test_annotations)

(5297, 4237, 1060)

In [11]:
vidPath = '/root/vids/scaled'

In [12]:
for item in tqdm(annotations):
    dst = item.split()[0]
    dst = os.path.join(video_path, dst)
    filename = dst.split('/')[-1]
    src = os.path.join(vidPath, filename)
    shutil.copy2(src, dst)

100%|██████████| 5297/5297 [00:01<00:00, 3167.63it/s]


In [19]:
# FIXME: does not seem to be working...
df_path = '/root/mmaction/third_party/dense_flow'
for cls in tqdm(classes.values()):
    src = os.path.join(video_path, cls)
    dst = os.path.join(raw_path, cls)
    command = f'python build_rawframes.py {src} {dst} --df_path {df_path}' + ' --level {1, 2}'
    os.system(command)

100%|██████████| 5/5 [00:00<00:00, 24.24it/s]
